In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random

# ---- Project Root ----
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parents[2]

DATASETS_DIR = PROJECT_ROOT / "datasets"
RESULTS_DIR = PROJECT_ROOT / "results"

FIGURES_DIR = RESULTS_DIR / "figures"
TABLES_DIR = RESULTS_DIR / "tables" / "backbone_benchmarking"
MODELS_DIR = PROJECT_ROOT / "models" / "backbone_benchmark_models"

TABLES_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

BNCI_PATH = DATASETS_DIR / "bnci_dataset" / "processed" / "preprocessed_BNCI.npz"

In [10]:
assert BNCI_PATH.exists(), f"BNCI file not found: {BNCI_PATH}"

d = np.load(BNCI_PATH, allow_pickle=True)
X = d["X"].astype(np.float32)
y = d["y"].astype(int)

print("Loaded BNCI:", X.shape)
print("Label distribution:", dict(zip(*np.unique(y, return_counts=True))))

Loaded BNCI: (640, 25, 561)
Label distribution: {np.int64(0): np.int64(160), np.int64(1): np.int64(160), np.int64(2): np.int64(160), np.int64(3): np.int64(160)}


In [11]:
class BNCI_Dataset(Dataset):
    def __init__(self, X, y, augment=False):
        self.X = X
        self.y = y
        self.augment = augment

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = self.X[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)

In [12]:
import torch
import torch.nn as nn

class EEGNet(nn.Module):
    def __init__(
        self,
        chans: int,
        samples: int,
        classes: int,
        F1: int = 8,
        D: int = 2,
        F2: int = 16,
        kern_len: int = 64,
        dropout: float = 0.5
    ):
        super().__init__()

        self.first = nn.Sequential(
            nn.Conv2d(1, F1, (1, kern_len), padding=(0, kern_len // 2), bias=False),
            nn.BatchNorm2d(F1),

            nn.Conv2d(F1, F1 * D, (chans, 1), bias=False),
            nn.BatchNorm2d(F1 * D),
            nn.ELU(),

            nn.AvgPool2d((1, 4)),
            nn.Dropout(dropout)
        )

        self.second = nn.Sequential(
            nn.Conv2d(F1 * D, F2, (1, 16), bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.AvgPool2d((1, 8))
        )

        # Dynamically compute flatten size
        with torch.no_grad():
            dummy = torch.zeros(1, 1, chans, samples)
            h = self.first(dummy)
            h = self.second(h)
            flatten_dim = h.view(1, -1).shape[1]

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flatten_dim, classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)              # (B, 1, C, T)
        x = self.first(x)
        x = self.second(x)
        return self.classifier(x)

In [13]:
class ShallowConvNet(nn.Module):
    def __init__(
        self,
        chans: int,
        samples: int,
        classes: int,
        F: int = 40,
        kern_len: int = 25,
        dropout: float = 0.5
    ):
        super().__init__()

        self.temporal = nn.Conv2d(
            1, F,
            (1, kern_len),
            padding=(0, kern_len // 2),
            bias=False
        )

        self.spatial = nn.Conv2d(
            F, F,
            (chans, 1),
            bias=False
        )

        self.pool = nn.Sequential(
            nn.ELU(),
            nn.AvgPool2d((1, 75)),
            nn.Dropout(dropout)
        )

        # Dynamic flatten size
        with torch.no_grad():
            dummy = torch.zeros(1, 1, chans, samples)
            h = self.temporal(dummy)
            h = self.spatial(h)
            h = self.pool(h)
            flatten_dim = h.view(1, -1).shape[1]

        self.classifier = nn.Linear(flatten_dim, classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.temporal(x)
        x = self.spatial(x)
        x = self.pool(x)
        x = x.flatten(1)
        return self.classifier(x)

In [14]:
class DeepConvNet(nn.Module):
    def __init__(
        self,
        chans: int,
        samples: int,
        classes: int,
        dropout: float = 0.5
    ):
        super().__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, 25, (1, 5), padding=(0, 2), bias=False),
            nn.Conv2d(25, 25, (chans, 1), bias=False),
            nn.BatchNorm2d(25),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(25, 50, (1, 5), padding=(0, 2), bias=False),
            nn.BatchNorm2d(50),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(50, 100, (1, 5), padding=(0, 2), bias=False),
            nn.BatchNorm2d(100),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(100, 200, (1, 5), padding=(0, 2), bias=False),
            nn.BatchNorm2d(200),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout)
        )

        # Dynamic flatten
        with torch.no_grad():
            dummy = torch.zeros(1, 1, chans, samples)
            h = self.block1(dummy)
            h = self.block2(h)
            h = self.block3(h)
            h = self.block4(h)
            flatten_dim = h.view(1, -1).shape[1]

        self.classifier = nn.Linear(flatten_dim, classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = x.flatten(1)
        return self.classifier(x)

In [15]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


def train_kfold(model_class, model_name, X, y):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"\n===== Training {model_name} on {device} =====")

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    model_save_dir = MODELS_DIR / model_name
    model_save_dir.mkdir(parents=True, exist_ok=True)

    fold_results = []

    for fold, (tr_idx, te_idx) in enumerate(kf.split(X, y), 1):

        print(f"\n--- Fold {fold} ---")

        Xtr, Xte = X[tr_idx], X[te_idx]
        ytr, yte = y[tr_idx], y[te_idx]

        ds_tr = BNCI_Dataset(Xtr, ytr)
        ds_te = BNCI_Dataset(Xte, yte)

        loader_tr = DataLoader(ds_tr, batch_size=32, shuffle=True)
        loader_te = DataLoader(ds_te, batch_size=64, shuffle=False)

        chans, samples = X.shape[1], X.shape[2]
        num_classes = len(np.unique(y))

        model = model_class(chans, samples, classes=num_classes).to(device)

        opt = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
        scheduler = ReduceLROnPlateau(opt, mode='max', factor=0.5, patience=3)
        loss_fn = nn.CrossEntropyLoss()

        best_acc = 0.0
        best_state = None
        patience = 8
        stale = 0

        for ep in range(1, 51):  

            model.train()
            train_loss = 0.0

            for xb, yb in loader_tr:
                xb, yb = xb.to(device), yb.to(device)

                opt.zero_grad()
                logits = model(xb)
                loss = loss_fn(logits, yb)
                loss.backward()
                opt.step()

                train_loss += loss.item()

            train_loss /= len(loader_tr)

            model.eval()
            ys, preds = [], []

            with torch.no_grad():
                for xb, yb in loader_te:
                    xb = xb.to(device)
                    logits = model(xb)

                    preds.extend(logits.argmax(dim=1).cpu().numpy())
                    ys.extend(yb.numpy())

            acc = accuracy_score(ys, preds)
            f1 = f1_score(ys, preds, average='weighted')

            scheduler.step(acc)

            if acc > best_acc + 1e-4:
                best_acc = acc
                best_state = {k: v.clone() for k, v in model.state_dict().items()}
                stale = 0
            else:
                stale += 1

            print(
                f"[{model_name} | Fold {fold} | Epoch {ep:02d}] "
                f"train_loss={train_loss:.4f} "
                f"val_acc={acc:.4f} "
                f"val_f1={f1:.4f} "
                f"(best={best_acc:.4f})"
            )

            if stale >= patience:
                print(f"Early stopping at epoch {ep}")
                break

        torch.save(
            best_state,
            model_save_dir / f"{model_name}_fold{fold}_best.pth"
        )

        print(f"Fold {fold} Best Accuracy: {best_acc:.4f}")

        fold_results.append(best_acc)

    print(f"\n===== {model_name} Mean Accuracy: {np.mean(fold_results):.4f} =====")

    return fold_results

In [ ]:
models = {
    "EEGNet": EEGNet,
    "ShallowConvNet": ShallowConvNet,
    "DeepConvNet": DeepConvNet
}

summary_rows = []

for name, model_class in models.items():

    fold_accs = train_kfold(model_class, name, X, y)

    summary_rows.append({
        "model": name,
        "acc_mean": float(np.mean(fold_accs)),
        "acc_std": float(np.std(fold_accs))
    })

summary_table = pd.DataFrame(summary_rows).sort_values("acc_mean", ascending=False)
summary_table.to_csv(TABLES_DIR / "bnci_deep_models_benchmark.csv", index=False)

print(summary_table)


===== Training EEGNet on cuda =====

--- Fold 1 ---
[EEGNet | Fold 1 | Epoch 01] train_loss=1.3843 val_acc=0.2578 val_f1=0.2135 (best=0.2578)
[EEGNet | Fold 1 | Epoch 02] train_loss=1.3284 val_acc=0.2969 val_f1=0.2531 (best=0.2969)
[EEGNet | Fold 1 | Epoch 03] train_loss=1.3051 val_acc=0.3438 val_f1=0.3426 (best=0.3438)
[EEGNet | Fold 1 | Epoch 04] train_loss=1.2734 val_acc=0.3203 val_f1=0.2981 (best=0.3438)
[EEGNet | Fold 1 | Epoch 05] train_loss=1.2643 val_acc=0.3438 val_f1=0.3154 (best=0.3438)
[EEGNet | Fold 1 | Epoch 06] train_loss=1.2304 val_acc=0.4141 val_f1=0.4115 (best=0.4141)
[EEGNet | Fold 1 | Epoch 07] train_loss=1.2245 val_acc=0.4219 val_f1=0.4167 (best=0.4219)
[EEGNet | Fold 1 | Epoch 08] train_loss=1.1838 val_acc=0.4141 val_f1=0.4114 (best=0.4219)
[EEGNet | Fold 1 | Epoch 09] train_loss=1.1565 val_acc=0.4141 val_f1=0.4107 (best=0.4219)
[EEGNet | Fold 1 | Epoch 10] train_loss=1.1223 val_acc=0.4141 val_f1=0.4078 (best=0.4219)
[EEGNet | Fold 1 | Epoch 11] train_loss=1.1109 